# Obtención de los Datos y Creación del Dataset

**Autor**: *Adrián Trujillo López*

**Modificado** por *Jorge Díaz Rosique*

**Fecha**: 21/09/2022

**Descripción**: Procesa los archivos de traza y log de un benchmark para crear los conjuntos de datos.

**Revisiones**:
* Revisión 0.01 - Archivo creado
* Revisión 0.02 - Estructura modificada y comentada
* Revisión 0.03 - Corrección errores en lifetimes
* Revisión 0.04 - Cambio a sharepoint

**Comentarios adicionales**: Basado en el código de David Ruiz


# Preeliminares

En este apartado se realizarán los procedimientos necesarios para poder trabajar posteriormente.

En primer lugar, se importarán las librerías necesarias.

In [ ]:
#### Importación de librerías ####

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import sys

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials

import shutil

En segundo lugar, se iniciará sesión en Google y se montará drive.

In [ ]:
# Autentificación de Drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Montar drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [9]:
# Se accede al Sharepoint
!wget https://unialicante-my.sharepoint.com/:f:/g/personal/sergio_mscloud_ua_es/NN4FT

--2022-09-21 17:01:45--  https://unialicante-my.sharepoint.com/:f:/g/personal/sergio_mscloud_ua_es/NN4FT
Resolving unialicante-my.sharepoint.com (unialicante-my.sharepoint.com)... 13.107.136.9, 13.107.138.9
Connecting to unialicante-my.sharepoint.com (unialicante-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112302 (110K) [text/html]
Saving to: ‘NN4FT’

NN4FT               100%[===================>] 109.67K   697KB/s    in 0.2s    

2022-09-21 17:01:46 (697 KB/s) - ‘NN4FT’ saved [112302/112302]



In [ ]:
# Se comprueba que esta montado con el siguiente comando de consola
!ls -l -h

total 8.0K
drwx------ 6 root root 4.0K Sep 11 11:24 drive
drwxr-xr-x 1 root root 4.0K Aug 31 13:47 sample_data


Por último se definen los nombres de los ficheros y los benchmarks a procesar.

In [ ]:
# define the name of the folders to process (it could be some folders per benchmark)
insertsort_folders = ['insertsort_111']
# define the benchmarks to process
benchmarks = [insertsort_folders]

# Creación de las Funciones Necesarias

Antes de comenzar con la propia extracción del `dataset`, se van a crear numerosas funciones que facilitarán el proceso.

## Funciones para la Extracción de Características

Como su propio nombre indica, en este apartado se definirán, en concreto, todas aquellas funciones, o variables, que se utilicen para la extracción de las características.

### Tiempo de Vida de los Registros

Todo lo necesario para el cálculo del `lifetime` de los diferentes registros.

En primer lugar, no se definirán funciones, se escribirán las expresiones regulares necesarias para la clasificación de instrucciones. Estas pueden ser:
* Read - Write
* Write - Read
* Read - Read

También se definirán la lista con los registros y varios conjuntos para la posterior evaluación de la creación del `dataset`.

In [ ]:
#### Expresiones regulares para la clasificación de instrucciones ####

# Obtención de las lineas de instrucción y cpsr
instrLine = r"^Info\s[0-9]"
cpsrLine = r"^\s{8}cpsr"

# Expresiones ya creadas con sufijos para facilitar la creación por instrucción
arm_cond = r"(eq|ne|cs|cc|mi|pl|vs|vc|hi|ls|ge|lt|gt|le){0,1}$"
arm_type = r"(d|b|sb|h|sh){0,1}"
addr_mode = r"(ia|ib|da|db|fd|fa|ed|ea){0,1}"
sflag = r"s{0,1}"

## Read Write instrucciones
ldm = r"^ldm"+addr_mode+arm_cond

## Write Read instrucciones
# Se definen las expresiones para mov, opsubadd, ldr, uxtb, logical, lsr
mov = r"^(mov|movw|movt|mvn)"+sflag+arm_cond
op = r"^(add|sub|mul|mla|rsb)"+sflag+arm_cond
ldr = r"^ldr"+arm_type+arm_cond
lsr = r"^(asr|lsl|lsr|ror|rrx)"+sflag+arm_cond
uxtb = r"^(uxtb|sbfx|ubfx)"+arm_cond
logical = r"^(and|orr|eor|bic|orn)"+sflag+arm_cond

# Se unen en una única expresión regular
writeReadInstr = r"^("+mov+"|"+op+"|"+ldr+"|"+lsr+"|"+uxtb+"|"+logical+")$"

## Read Read instrucciones
# Se definen las expresiones para branch, testop, str1, stm
str1 = r"^str"+arm_type+arm_cond
stm = r"^stm"+addr_mode+arm_cond
branch = r"^(b|bl|bx|blx)"+arm_cond
testop = r"^(tst|teq)"+arm_cond
compare = r"^(cmp|cmn)"+arm_cond
pld = r"^(pld|pldw|pli)"+arm_cond

# Se unen en una única expresión regular
readReadInstr = r"^("+str1+"|"+stm+"|"+branch+"|" +testop+"|"+compare+"|"+pld+")$"

## Se crean las variables necesarias finales

# Lista de todos los registros
registers = ["r0","r1","r2","r3","r4","r5","r6",
             "r7","r8","r9","sl","fp","ip","sp","lr"]

# Expresiones regulares para las intrucciones de acceso, lectura y escritura
memoryAccesInstr = r"^("+str1+"|"+stm+"|"+ldr+"|"+ldm+"|"+pld+")$"
memoryReadInstr = r"^("+ldr+"|"+ldm+"|"+pld+")$"
memoryWriteInstr = r"^("+str1+"|"+stm+")$"

#test = "lsl"
#if re.search(doInstr,test) and (test not in exInstr):
#  print("True")

## Para las instrucciones condicionales

# Variables para detectar instrucciones condicionales y eliminar falsos positivos
doInstr = r"(eq|ne|cs|cc|mi|pl|vs|vc|hi|ls|ge|lt|gt|le)$"
exInstr = ["teq","movs","svc","lsls","bics","muls"]

In [ ]:
# Conjuntos para recoger la información
setCond = set()
setWR = set()
setRR = set()
setRW = set()
setMemory = set()
setMemoryRead = set()
setMemoryWrite = set()
setIgnoredInstr = set()
setInstructions = set()

Tras esto, en segundo lugar se definirá la función `getInformationProgram()`. Esta pretende obtener la información del programa leyendo la traza linea a linea, viendo a que grupo pertenece la instrucción y devolviendo el tiempo de vida de los registros y el núero registros a los que se ha accedido, leido o escrito.

Sin embargo, antes de esto se necesitará definir una función que obtengan las instrucciones condicionales utilizadas utilizada. Esta será la siguiente, `executedInstr()`. Para ello se utilizará el `cpsr`.

In [ ]:
def executedInstr(cond, cpsr):
  binarynum = bin(cpsr)[2:]
  
  try:
    N = int(binarynum[31])
  except IndexError:
    N = 0
  try:
    Z = int(binarynum[30])
  except IndexError:
    Z = 0
  try:
    C = int(binarynum[29])
  except IndexError:
    C = 0
  try:
    V = int(binarynum[28])
  except IndexError:
    V = 0

  if(cond=="eq"):
    if Z == 1:
      return True
  elif(cond=="ne"):
    if Z == 0:
      return True
  elif(cond=="cs"):
    if C == 1:
      return True
  elif(cond=="cc"):
    if C == 0:
      return True
  elif(cond=="mi"):
    if N == 1:
      return True
  elif(cond=="pl"):
    if N == 0:
      return True
  elif(cond=="vs"):
    if V == 1:
      return True
  elif(cond=="vc"):
    if V == 0:
      return True
  elif(cond=="hi"):
    if (C == 1) and (Z == 0):
      return True
  elif(cond=="ls"):
    if (C == 0) and (Z == 1):
      return True
  elif(cond=="ge"):
    if N == V:
      return True
  elif(cond=="lt"):
    if N != V:
      return True
  elif(cond=="gt"):
    if (Z == 0) and (N == V):
      return True
  elif(cond=="le"):
    if (Z == 1) and (N != V):
      return True
  return False

In [ ]:
def getInformationProgram():
  
  ## Inicialización de las variables y definición de funciones

  # Estas dos funciones se utilizarán para ir aumentando los lifetimes de cada
  # uno de los registros. Se utilizará una u otra en función del tipo de ins-
  # trucción proporcionada.
  def calculateLTRegRead(read):
    for reg in registers:
      if (re.match(stm,instr)):
        if reg in read and values[reg]!=0:
          lifetime[reg]+=index-values[reg]
          values[reg]=index
          memoryZero[reg] = 0
        elif reg in read and values[reg]==0:
          memoryZero[reg] = 1
      else:
        if reg in read:
          lifetime[reg]+=index-values[reg]
          values[reg]=index
      
  def calculateLTRegWritten(written):
    for reg in registers:
      if (re.match(ldm,instr)):
        if reg in written and memoryZero[reg]==0:
          values[reg]=index
        elif reg in written and memoryZero[reg]==1:
          values[reg]=0
      else:
        if reg in written:
          values[reg]=index
        
  # Conteo para acceso, lectura y escritura
  memoryAccess = 0
  memoryRead = 0
  memoryWrite = 0
  
  lifetime = {} # Diccionario que almacenará el lifetime de cada registro
  values = {} # Diccionario que almacenará la posición de la última instrucción
  memoryZero = {}
  for i in registers:
    values[i] = 0
    lifetime[i] = 0
    memoryZero[i] = 0

  ## Bucle de extracción de información, tiempo
  index = 0
  # Para cada fila de la traza se obtiene la instrucción que se usa y se
  # comprueba su tipo. Se extrae la información utilizando las expresiones 
  # regulares definidas anteriormente y se va incrementando el lifetime para 
  # cada registro utilizado en estas.
  for row in df.itertuples(index=True, name="line"):
    instr = row[1]
    index = row[0]
    
    setInstructions.add(instr)
    # Para detectar las instrucciones condicionales
    condMatch = re.search(doInstr,instr)
    if(condMatch and (instr not in exInstr)):
      setCond.add(instr)
      if not executedInstr(condMatch.group(0), row[4]):
        continue
        
    if(re.match(writeReadInstr,instr)): # Write Read
      setWR.add(instr)
      calculateLTRegRead(row[3])
      calculateLTRegWritten(row[2])
    elif(re.match(readReadInstr,instr)): # Read Read
      setRR.add(instr)
      calculateLTRegRead(row[2])
      calculateLTRegRead(row[3])
    elif(re.match(ldm,instr)): # Read Write (ldm)
      setRW.add(instr)
      calculateLTRegRead(row[2])
      calculateLTRegWritten(row[3])
    else:
      setIgnoredInstr.add(instr)

    if(re.match(memoryAccesInstr,instr)):
      memoryAccess += 1
      setMemory.add(instr)
      if(re.match(memoryReadInstr,instr)): 
        memoryRead += 1
        setMemoryRead.add(instr)
      if(re.match(memoryWriteInstr,instr)):
        memoryWrite += 1
        setMemoryWrite.add(instr)

  return lifetime, memoryAccess, memoryRead, memoryWrite

### Traza de Ejecución

Esta muesta las instrucciones que se han ido ejecutando y, también, los valores que tienen los registros después de cada instrucción. Con la función `createdDataframe()` se obtendrá un `.csv`con cuatro columnas:
1. Instrucción ejecutada
2. Primer parámetro de la instrucción
3. Segundo parámetro de la instrucción
4. Valor de `cpsr`

In [ ]:
def createDataframe(trace_file):
  # Se crea el archivo para editar
  fileToWrite = open("traceFile.csv", "w")

  # Se abre el archivo para leer
  with open(trace_file) as file:

    # Para cada linea del fichero se obtendrán los cuatro parámetros mencionados
    for line in file:
      if re.match(instrLine,line):
        columns = line.split()
        fileToWrite.write('%s;' % (columns[5])) # Instrucción
        if(len(columns)>6):
          registersAB = columns[6].split(",",1)
          if(len(registersAB)==1):
            fileToWrite.write('%s;%s;' % (registersAB[0], "-")) # Registro A
          else:
            fileToWrite.write('%s;%s;' % (registersAB[0], registersAB[1])) # Registro A y B
      elif re.match(cpsrLine,line):
        columns = line.split()
        fileToWrite.write('%s\n' % (columns[2]))
  fileToWrite.close()
  
  # Se añade la columna de cpsr al .csv creado
  df = pd.read_csv("traceFile.csv", sep=";", names=["Instruction","A","B","cpsr"])
  df.cpsr = df.cpsr.shift(1).fillna(0)
  df.cpsr = df.cpsr.astype(int)
  
  return df

### Tamaño de las Cabeceras

Simplemente, se obtiene el tamaño de las secciones `.text`, `.data`, `.bss` y `.rodata`.

In [ ]:
def getSizePrograms(folder, info_db):
  
  # Se busca cada tamaño dentro de info_db
  ind_df = info_db.loc[info_db['ind'] == folder]

  text_size = ind_df['text_size'].values[0]
  data_size = ind_df['data_size'].values[0]
  bss_size = ind_df['bss_size'].values[0]
  stack_size = ind_df['stack_size'].values[0]

  return text_size, data_size, bss_size, stack_size

### Tolerancia a Fallos

La función `getCoverage()` busca obtener cuando se produce alguna anomalía en algún valor se obtenga un fallo de tipo SDC o un fallo de tipo Hang.

In [ ]:
def getCoverage(folder, info_db):

  # Se inicializan las variables
  rf_sdc = 0.0
  rf_hang = 0.0

  sdc_fields = ["r0_sdc","r1_sdc","r2_sdc","r3_sdc","r4_sdc","r5_sdc","r6_sdc","r7_sdc","r8_sdc","r9_sdc","r10_sdc","r11_sdc","r12_sdc","sp_sdc","lr_sdc","pc_sdc"]
  hang_fields = ["r0_hang","r1_hang","r2_hang","r3_hang","r4_hang","r5_hang","r6_hang","r7_hang","r8_hang","r9_hang","r10_hang","r11_hang","r12_hang","sp_hang","lr_hang","pc_hang"]

  # Se busca cada error dentro de info_db
  ind_df = info_db.loc[info_db['ind'] == folder]

  for field in sdc_fields:
    rf_sdc += ind_df[field].values[0]

  for field in hang_fields:
    rf_hang += ind_df[field].values[0]

  # Sobre 1
  rf_sdc = rf_sdc/16
  rf_hang = rf_hang/16

  return rf_sdc, rf_hang


### Obtención de características

Por último, se pueden obtener diversas características del dataframe `info_db` directamente con la función `getCharacteristics()`.

In [ ]:
def getCharacteristics(folder, info_db):
  
  # Se busca cada característica dentro de info_db
  ind_df = info_db.loc[info_db['ind'] == folder]

  size = ind_df['size'].values[0]
  cycles = ind_df['cycles'].values[0]
  ft_overal = ind_df['ft_overall'].values[0]
  sdc_overall = ind_df['sdc_overall'].values[0]
  hang_overall = ind_df['hang_overall'].values[0]


  return size, cycles, ft_overal, sdc_overall, hang_overall

## Funciones para Debug

Se definirá una función que mostrará la información del proceso realizado, `printInf()`.


In [ ]:
def printInf():
    print("Cond: ",setCond)
    print("WR: ",setWR)
    print("RR: ",setRR)
    print("RW: ",setRW)
    print("MemroyInstr: ",setMemory)
    print("MemroyRead: ",setMemoryRead)
    print("MemroyWrite: ",setMemoryWrite)
    print("IgnoredInstr: ",setIgnoredInstr)
    print("All Instructions:",setInstructions)
    print("------------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("Instrucciones totales:", df.shape[0])
    print("Accesos a memoria:", memoryAccess)
    print("Lecturas a memoria:", memoryRead)
    print("Escrituras a memoria:", memoryWrite)

# Proceso

Tras lo realizado en los apartados anteriores, es el momento de unirlo todo. Es decir, se generará el dataset de información aplicando las funciones definidas anteriormente para cada benchmark.

In [ ]:
#### Creación del dataset ####

## Inicialización de las variables

csv_extension = '_db_info.csv' # Extensión .csv
database_extension = '_db' # Extesión de la base de datos
global_path = '/content/drive/MyDrive/ML4FT/data/raw_data/' # Ruta de los datos
all_csv = [] # Lista donde irán todos los .csv de cada benchmarks
folder_done = []

## Comienza el bucle

for bench in benchmarks:
  print("In " + bench[0][0:-4] + ": ")
  # Se crea un csv para cada uno de los benchmarks
  csv = bench[0][0:-4] + '.csv'
  all_csv.append(csv)

  # Se abre el .csv y se nombran las diferentes columnas
  with open(csv, "w") as f:
    f.write('bench;ind;r0;r1;r2;r3;r4;r5;r6;r7;r8;r9;sl;fp;ip;sp;lr;totalInstructions;memoryRead;memoryWrite;memoryAccess;text;data;bss;stack;rf_sdc;rf_hang;size;cycles;ft_overall;sdc_overall;hang_overall\n')
    f.close()

  # Para codigo de cada bench se obitenen todos los ficheros que lo contienen
  for code in bench:
    print("> Analyzing " + code + " ...")
    bench_path = global_path + code
    info_path = bench_path + '/' +  code + csv_extension # Archivo con información individual
    info_db = pd.read_csv(info_path, sep=",")
    database_path = bench_path + database_extension # Archivo con todos los individuos

    # Se obtiene la lista de todos los indiviudos y se muestra
    folder_list = os.listdir(database_path)
    for x in range(len(folder_list)):
      print("  · ",folder_list[x])

    # Se hacen los preparativos para recorrerla
    total_ind = len(folder_list)
    i = 0

    # Se recorre cada uno de ellos comprobando que no se ha realizado ya ese.
    # Tras esto se obtiene la información necesaria aplicando cada una de las
    # funciones definidas anteriormente.
    for folder in folder_list:
      if folder not in folder_done: # OFlags deben ser unicos
        # Se obtiene la ruta y se inicializan o modifican las variables
        folder_done.append(folder)
        outputToWrite = ""
        i+=1
        trace_file = database_path + '/' + folder + '/' + code + '.ARM.elf.trace'

        ## Se aplican las funciones

        # Se inicializa el DataFrame
        df = createDataframe(trace_file)
        # Se obtiene la información del programa
        lifetime, memoryAccess, memoryRead, memoryWrite = getInformationProgram()
        # Se obtiene el tamaño del programa
        text_size, data_size, bss_size, stack_size = getSizePrograms(folder, info_db)
        # Se obtienen los errores
        rf_sdc, rf_hang = getCoverage(folder, info_db)
        # Se obtienen las características del df
        size, cycles, ft_overall, sdc_overall, hang_overall = getCharacteristics(folder, info_db)
      
        ## Se completa la fila a añadir en el .csv

        # bench y fichero
        outputToWrite += bench[0][0:-4] + ';' + folder + ';'

        # lifetime de los registros
        for reg in registers:
          outputToWrite += str(lifetime[reg])+";"

        # información obtenida
        outputToWrite += str(df.shape[0])+";"+str(memoryRead)+";"+str(memoryWrite)+";"+str(memoryAccess)+";"+str(text_size)+";"+str(data_size)+";"+str(bss_size)+ \
        ";"+str(stack_size)+";"+str(rf_sdc)+";"+str(rf_hang)+";"+str(size)+";"+str(cycles)+";"+str(ft_overall)+";"+str(sdc_overall)+";"+str(hang_overall)+"\n"

        # Se escribe en le fichero para el bench
        with open(csv, "a") as f:
          f.write(outputToWrite)
          f.close()
        
        print("- Ind: ", i, "/", total_ind)

  #Se guarda cada uno de los .csv
  print('Saving ' + csv + ' ...')
  save_path = '/content/drive/MyDrive/ML4FT/data/datasets/'
  shutil.move(csv, save_path)
  print('Done ' + csv + ' ...\n')

In insertsort: 
> Analyzing insertsort_111 ...
  ·  c9ec94a9aaf8eb3b960b8459729656b1a6ae431d
  ·  abd6433bf9922968750bfc7ab120465f03757571
  ·  751a547b7d610fec0001f4a8302a1646926b74f1
  ·  d41445f5e9b26d5a03ebb2cc1873b2016ae3c315
  ·  b2d852c0879ade9ae66cd3d4ddc82e8081cceb84
  ·  7398e2b0ffd7b6d37518d309f097c37fb4a7911b
- Ind:  1 / 6
- Ind:  2 / 6
- Ind:  3 / 6
- Ind:  4 / 6
- Ind:  5 / 6
- Ind:  6 / 6
Saving insertsort.csv ...
Done insertsort.csv ...



La siguiente celda solo es necesaria si se desean copiar los datasets en alguna ruta en concreto.

In [ ]:
# Mueve los datasets a cualquier directorio si no están guardados ya allí
#save_path = '/content/drive/MyDrive/ML4FT/data/datasets/'
#for file in all_csv:
#  shutil.move(file, save_path)

Por último, se muestra la información para debug.

In [ ]:
# debug
printInf()

Cond:  {'addne', 'strhi', 'streq', 'ldmeq', 'bls', 'blt', 'bhi', 'bmi', 'strlt', 'strne', 'movne', 'bge', 'ldmne', 'movgt', 'bgt', 'addle', 'ldrne', 'bxeq', 'beq', 'addeq', 'ble', 'moveq', 'ldrhi', 'bne', 'movlt'}
WR:  {'asr', 'bic', 'uxtb', 'and', 'addle', 'asrs', 'add', 'subs', 'mov', 'sub', 'movw', 'ldrb', 'movt', 'lsr', 'addeq', 'moveq', 'movlt', 'orr', 'lsl', 'ldr', 'movs'}
RR:  {'str', 'strb', 'streq', 'bx', 'cmn', 'stm', 'bl', 'ble', 'stmdb', 'b', 'blt', 'bxeq', 'cmp', 'blx', 'strlt', 'tst', 'beq'}
RW:  {'ldmib', 'ldm', 'ldmeq'}
MemroyInstr:  {'str', 'strb', 'streq', 'stm', 'stmdb', 'ldmeq', 'ldm', 'ldrb', 'strlt', 'ldmib', 'ldr'}
MemroyRead:  {'ldmeq', 'ldm', 'ldrb', 'ldmib', 'ldr'}
MemroyWrite:  {'str', 'stm', 'strb', 'streq', 'stmdb', 'strlt'}
IgnoredInstr:  {'svc', 'INTERCEPT'}
All Instructions: {'str', 'addne', 'strhi', 'streq', 'ldmeq', 'bls', 'asr', 'bhi', 'blt', 'blx', 'bic', 'ldm', 'bmi', 'strlt', 'strne', 'uxtb', 'movne', 'and', 'cmn', 'bl', 'b', 'bge', 'ldmne', 'movgt

# Celdas de pruebas

A continuación, celdas comentadas que solo se han utilizado en pruebas.

In [ ]:
#drive = GoogleDrive(gauth)
#local_download_path = '/content/drive/MyDrive/Campañas de inyección de fallos'

#try:
#    os.makedirs(local_download_path)
#except: pass

#folder_list = drive.ListFile({'q': "'1geofFF81aABquHgYTvSkVugIDI0cY6P3' in parents"}).GetList()

#Write headers
#with open("insertsort_101.csv", "w") as f:
#  f.write('folder;r0;r1;r2;r3;r4;r5;r6;r7;r8;r9;sl;fp;ip;sp;lr;totalInstructions;memoryRead;memoryWrite;memoryAccess;.text;.data;.bss;.rodata;unAce;SDC;Hang\n')
#  f.close()

#print(len(folder_list))

In [ ]:
#!size program.elf

In [ ]:
#!ls -l

In [ ]:
pd.read_csv(save_path+csv, sep = ';')

,bench,ind,r0,r1,r2,r3,r4,r5,r6,r7,...,data,bss,stack,rf_sdc,rf_hang,size,cycles,ft_overall,sdc_overall,hang_overall
0,insertsort,c9ec94a9aaf8eb3b960b8459729656b1a6ae431d,415,344,468,648,705,586,530,767,...,2480,308,1025,10.9375,23.6250,145202,548,72.909091,8.681818,18.409091
1,insertsort,abd6433bf9922968750bfc7ab120465f03757571,260,267,483,562,288,589,449,686,...,2480,308,1025,11.0625,22.0000,145960,467,74.272727,8.681818,17.045455
2,insertsort,751a547b7d610fec0001f4a8302a1646926b74f1,265,277,493,520,610,600,460,697,...,2480,308,1025,12.3125,20.9375,145586,478,74.045455,9.454545,16.500000
3,insertsort,d41445f5e9b26d5a03ebb2cc1873b2016ae3c315,420,330,468,578,703,586,534,771,...,2480,308,1025,9.1875,22.3750,145202,552,75.000000,7.454545,17.545455
4,insertsort,b2d852c0879ade9ae66cd3d4ddc82e8081cceb84,409,211,558,636,675,648,569,806,...,2480,308,1025,11.0625,22.0000,145960,467,74.272727,8.681818,17.045455
5,insertsort,7398e2b0ffd7b6d37518d309f097c37fb4a7911b,1817,334,1011,1213,2034,1993,1853,2090,...,2480,308,1025,5.9375,16.5000,145394,1871,81.318182,5.318182,13.363636
